
## Creating a fingerprint from a pcap file

In this application, it is aimed to produce device fingerprints from  [pcap files](https://en.wikipedia.org/wiki/Pcap) using python language and [Scapy](https://scapy.net/) library. Two articles were used in this application: ([Miettinen at al.](https://ieeexplore.ieee.org/document/7980167/) and [Bezawada at al](https://dl.acm.org/citation.cfm?id=3266452)


The first of these articles, IOT SENTINEL, created the fingerprint using the features in table 1. 
 
Of these properties, the four are integers( ```Packet Size ```,```Destination IP counter ```,```Source port```,```Destination port ``` ) and the remaining 19 are binary values. These binary values determine if a protocol was used in a package ( 1 = in use 0 = not in use ).

| Type | Features |
| ------ | ------ |
|Link layer protocol (2)|  ARP / LLC|
|Network layer protocol (4)  |IP / ICMP / ICMPv6 / EAPoL|
| Transport layer protocol (2) | TCP / UDP|
| Application layer protocol (8)  |HTTP / HTTPS / DHCP / BOOTP / SSDP / DNS / MDNS / NTP|
| IP options (2) | Padding / RouterAlert|
| Packet content (2)  |Size (int) / Raw data|
| IP address (1) |Destination IP counter (int)|
| Port class (2) | Source (int) / Destination (int)|

| **Table 1**|
| ------ |
                                                    

[The codes of the IotSENTINEL](https://github.com/andypitcher/IoT_Sentinel) program that make up this study are publicly available. However, this program has some disadvantages:

- Written in Python 2.
- The dpkt library was used. (Whether it is advantageous or disadvantage is debatable)
- It contains some errors. For example:
    - The padding value is initially set to 0 and is not used anywhere in the program. However, this superfluous variable is written to the CSV file and shown as a feature.
    - The IP value is mentioned in the article and the program contains this value, but this variable is not included in the CSV result file




In the second study, [Bezawada et al.](https://dl.acm.org/citation.cfm?id=3266452) use many protocol-based features of IotSENTINEL study, indicating that they are pleasant features that reflect device behaviour (Such as ARP, SSL, LLC, EAPOL, HTTP, MDNS and DNS, 17 properties in total). On the other hand, they removed IP addresses and the target and source port numbers because they thought these features are too specific and far from reflecting the behaviour. In addition to these features, 3 additional features from the payload are included in the fingerprint:  entropy of payload, TCP payload length and TCP window size. The features used are listed in Table 2.

|	Feature taken from IotSENTINEL	|The features of Bezawada et al. |
|:---  |		:---  |
|ARP	|Entropy of payload|
|	IP/ICMP/ICMPv6/EAPoL	|TCP payload length|
|	TCP/UDP	|TCP window size|
|	HTTP/HTTPS/DHCP/BOOTP/SSDP/DNS/MDNS/NTP	|-|
|	Padding/Router Alert	|-|

| **Table 2**|
| ------ |



However, we do not agree with Bezawada et al. about IP addresses and port numbers features. This is because IoTSENTINEL does not use IP addresses and Port numbers directly, but a digest features after counting process for IP addresses and classification process for Port numbers. With this method, they avoid unnecessary information such as IP address and port number, and they obtain important information about two critical issues, such as IP address and Port number diversity.


Another point is that in this study, the packet size property is not used and a new property named TCP payload length is defined. There is already a very serious correlation between TCP payload length and packet size (TCP payload length = packet size - header sizes). Moreover, unlike TCP payload length, the packet size feature provides information about all packets, not just the TCP  packets. 

Considering all this information, in our fingerprint study, we will use all the features of the IoTSENTINEL study and two features (```Entropy of payload``` and `TCP window size`) from the study of Bezawada et al.


### Referances

1. 	B. Bezawada, M. Bachani, J. Peterson, H. Shirazi, I. Ray, and I. Ray, ["Iotsense: Behavioral fingerprinting of iot devices," ](https://ieeexplore.ieee.org/document/7980167/) arXiv preprint arXiv:1804.03852, 2018.

2.	M. Miettinen, S. Marchal, I. Hafeez, N. Asokan, A.-R. Sadeghi, and S. Tarkoma, ["IoT Sentinel: Automated device-type identification for security enforcement in IoT,"](https://dl.acm.org/citation.cfm?id=3266452) in 2017 IEEE 37th International Conference on Distributed Computing Systems (ICDCS), 2017, pp. 2177-2184: IEEE.



### Input & Output

`Input Files`: All files with the pcap extension in the “./pcaps/” folder is read.

`Output Files`: csv result files with the same name as the processed pcap files are saved to "./csvs/" folder.

In [14]:
from scapy.all import*
import math
import os


## ↑
In this section the libraries to be used are imported. 


### =================================================================
In this section, all pcap files in the `pcaps` folder are found and their addresses are assigned to the variable named files_add.
# &darr;  
 

In [15]:
def find_the_way(path,file_format):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                files_add.append(os.path.join(r, file))  
    return files_add
files_add=find_the_way('./pcaps/','.pcap')

In this section, a folder named `csvs` is created to save the result files.
# ↓ 

In [16]:
def folder(f_name):
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")
folder_name="./csvs/"
folder(folder_name)

        

Port numbers are classified in this part.

| Port Numbers | Equivalents |
| :------ | :------ |
|No port| 0|
|Well known ports (between 0 and 1023) |1|
|Rregistered ports (between 1024 and 49151)  |2|
|Dynamic ports (between  49152 and 65535) |3|
# ↓ 

In [17]:

def port_class(port):
    if 0 <= port <= 1023:
        return 1
    elif  1024 <= port <= 49151 :
        return 2
    elif 49152 <=port <= 65535 :
        return 3
    else:
        return 0


In [18]:
def pre_entropy():
    
    characters=[]
    for i in payload:
            characters.append(i)
    return shannon(characters)
            

Calculating the entropy value. [This study](https://github.com/batidiane/Network-Entropy) was used for the calculation. 


# ↓ 

In [19]:
def shannon(data):
    LOG_BASE = 2
   # We determine the frequency of each byte
   # in the dataset and if this frequency is not null we use it for the
   # entropy calculation
    dataSize = len(data)
    ent = 0.0
    freq={} 
    for c in data:
        if c in freq:
            freq[c] += 1
        else:
            freq[c] = 1
   # to determine if each possible value of a byte is in the list
    for key in freq.keys():
        f = float(freq[key])/dataSize
        if f > 0: # to avoid an error for log(0)
            ent = ent + f * math.log(f, LOG_BASE)
    return -ent

This section is the main backbone of our program. Unfortunately, because it includes for loops, we cannot make a modular narrative by dividing it into parts. So, we will explain the whole block here.


Another problem with Scripting is that too many of the `try: except` blocks are used. Unfortunately, if the result of queries in the `Scapy` library is negative, the program crashes (For example, if you query whether a package that does not contain a port number contains a port number, the program crashes). So we used  `try: accept:` blocks to get the program running.


In this section, the following operations are performed briefly.


- The pcap_files variable contains the addresses of the pcap files. The `for` loop moves through the values of this variable, allowing all files to be processed.


- The second for loop examines individual packets in the processed pcap file.


- All properties are initially assigned a value of 0. These properties are then queried in the package. If the properties have corresponding data, the data is processed in the variable, otherwise, the value of the variable remains as 0.


- At the end of the second `for` loop, 25 properties are obtained for a packet. In addition to this property, the name of the file is added with the tag name as the 26th Property and written to the CSV file as 1 line. Each packet is processed as a new line in this file.


- When the second `for` loop ends, the corresponding CSV file is closed and then the next file is processed.

In [20]:
count=0
for i in files_add:
    pkt = rdpcap(i)
    print("\n\n"+"====================="+ i[8:]+"====================="+"\n" )
    print(pkt)
    csvname=str(i)
    csvname=csvname[8:-5]
    
    csvname=csvname.replace("\\","@")
    label=csvname
    where_is_at=label.find("@")
    label=label[0:where_is_at]
    
    ths = open(folder_name+csvname+".csv", "w")    

    
    

    ip_add_count = 0    #
    dst_ip_list=[]
    
    
    
    for j in pkt:
        count=count+1
        
        layer_2_arp = 0
        layer_2_llc = 0
        
        layer_3_eapol = 0        
        layer_3_ip = 0
        layer_3_icmp = 0
        layer_3_icmp6 = 0

        
        
        layer_4_tcp = 0
        layer_4_udp = 0
        layer_4_tcp_ws=0
        
        
        layer_7_http = 0
        layer_7_https = 0
        layer_7_dhcp = 0
        layer_7_bootp = 0
        layer_7_ssdp = 0
        layer_7_dns = 0
        layer_7_mdns = 0
        layer_7_ntp = 0

        ip_padding = 0
        ip_ralert = 0


        port_class_src = 0
        port_class_dst = 0

        pck_size = 0
        pck_rawdata = 0
        entropy=0


        

        
        try:
            pck_size=j.len
        except:pass
    
        try:
            if j[IP]:
                layer_3_ip = 1
                temp=str(j[IP].show)
                if "ICMPv6" in temp:
                    layer_3_icmp6 = 1
                    
            temp=str(j[IP].dst)
            if temp not in dst_ip_list:
                ip_add_count=ip_add_count+1
                dst_ip_list.append(temp)
            port_class_src = port_class(j[IP].sport)
            port_class_dst = port_class(j[IP].dport)
                
        except:pass 
        
        
    
        try:
            if j[IP].ihl >5:
                if IPOption_Router_Alert(j):
                    pad=str(IPOption_Router_Alert(j).show)
                    if "Padding" in pad:
                        ip_padding=1
                    ip_ralert = 1     
        except:pass 
        
        try:
            if j[ICMP]:
                layer_3_icmp = 1  
        except:pass 
        
        try:
            if j[Raw]:
                pck_rawdata = 1   
        except:pass 
        
        
        try:
            if j[UDP]:
                layer_4_udp = 1
                if j[UDP].sport==68 or j[UDP].sport==67:
                    layer_7_dhcp = 1
                    layer_7_bootp = 1
                if j[UDP].sport==53 or j[UDP].dport==53:
                    layer_7_dns = 1      
                if j[UDP].sport==5353 or j[UDP].dport==5353:
                    layer_7_mdns = 1                    
                if j[UDP].sport==1900 or j[UDP].dport==1900:
                    layer_7_ssdp = 1                    
                if j[UDP].sport==123 or j[UDP].dport==123:
                    layer_7_ntp = 1                    
        except:pass 
                
        
        try:
            if j[TCP]:

                layer_4_tcp = 1
                layer_4_tcp_ws=j[TCP].window
                if j[TCP].sport==80 or j[TCP].dport==80:
                    layer_7_http = 1      
                if j[TCP].sport==443 or j[TCP].dport==443:
                    layer_7_https = 1  
        except:pass        
        
        try:
            if j[ARP]:
                layer_2_arp = 1                                 
        except:pass 
                        
        
        try:
            if j[LLC]:
                layer_2_llc = 1                                 
        except:pass 
                        
        
        
        try:
            if j[EAPOL]:
                layer_3_eapol = 1                                 
        except:pass 
            
        try:
            entropy=pre_entropy(j[Raw].original)
        except:pass
        line=[layer_2_arp, layer_2_llc, layer_3_eapol, layer_3_ip, layer_3_icmp, layer_3_icmp6, layer_4_tcp, layer_4_udp, layer_4_tcp_ws, layer_7_http, layer_7_https, layer_7_dhcp, layer_7_bootp, layer_7_ssdp, layer_7_dns, layer_7_mdns, layer_7_ntp, ip_padding, ip_add_count, ip_ralert, port_class_src, port_class_dst, pck_size, pck_rawdata,entropy, label]  
        line=str(line).replace("[","")
        line=str(line).replace("]","")
        line=str(line).replace("\'","")
        ths.write(str(line)+"\n")  
    ths.close()          
            



=====================Aria\Setup-A-1-STA.pcap=====================

<Setup-A-1-STA.pcap: TCP:30 UDP:10 ICMP:0 Other:9>


=====================Aria\Setup-A-2-STA.pcap=====================

<Setup-A-2-STA.pcap: TCP:30 UDP:6 ICMP:0 Other:7>


=====================Aria\Setup-A-3-STA.pcap=====================

<Setup-A-3-STA.pcap: TCP:30 UDP:10 ICMP:0 Other:8>


=====================Aria\Setup-A-4-STA.pcap=====================

<Setup-A-4-STA.pcap: TCP:30 UDP:10 ICMP:0 Other:8>


=====================Aria\Setup-A-5-STA.pcap=====================

<Setup-A-5-STA.pcap: TCP:30 UDP:10 ICMP:0 Other:8>


=====================Aria\Setup-C-1-STA.pcap=====================

<Setup-C-1-STA.pcap: TCP:30 UDP:12 ICMP:0 Other:8>


=====================Aria\Setup-C-10-STA.pcap=====================

<Setup-C-10-STA.pcap: TCP:30 UDP:10 ICMP:0 Other:8>


=====================Aria\Setup-C-11-STA.pcap=====================

<Setup-C-11-STA.pcap: TCP:30 UDP:10 ICMP:0 Other:8>


=====================Aria\Setup-C-1



=====================D-LinkDoorSensor\Setup-C-16-STA.pcap=====================

<Setup-C-16-STA.pcap: TCP:185 UDP:0 ICMP:0 Other:6>


=====================D-LinkDoorSensor\Setup-C-17-STA.pcap=====================

<Setup-C-17-STA.pcap: TCP:217 UDP:2 ICMP:0 Other:8>


=====================D-LinkDoorSensor\Setup-C-18-STA.pcap=====================

<Setup-C-18-STA.pcap: TCP:279 UDP:7 ICMP:0 Other:6>


=====================D-LinkDoorSensor\Setup-C-19-STA.pcap=====================

<Setup-C-19-STA.pcap: TCP:172 UDP:0 ICMP:0 Other:4>


=====================D-LinkDoorSensor\Setup-C-2-STA.pcap=====================

<Setup-C-2-STA.pcap: TCP:136 UDP:8 ICMP:0 Other:8>


=====================D-LinkDoorSensor\Setup-C-20-STA.pcap=====================

<Setup-C-20-STA.pcap: TCP:171 UDP:0 ICMP:0 Other:8>


=====================D-LinkDoorSensor\Setup-C-3-STA.pcap=====================

<Setup-C-3-STA.pcap: TCP:168 UDP:2 ICMP:0 Other:8>


=====================D-LinkDoorSensor\Setup-C-4-STA.pcap========



=====================D-LinkSiren\Setup-C-18-STA.pcap=====================

<Setup-C-18-STA.pcap: TCP:465 UDP:96 ICMP:0 Other:23>


=====================D-LinkSiren\Setup-C-19-STA.pcap=====================

<Setup-C-19-STA.pcap: TCP:466 UDP:128 ICMP:0 Other:32>


=====================D-LinkSiren\Setup-C-2-STA.pcap=====================

<Setup-C-2-STA.pcap: TCP:463 UDP:96 ICMP:0 Other:29>


=====================D-LinkSiren\Setup-C-3-STA.pcap=====================

<Setup-C-3-STA.pcap: TCP:464 UDP:112 ICMP:0 Other:33>


=====================D-LinkSiren\Setup-C-4-STA.pcap=====================

<Setup-C-4-STA.pcap: TCP:464 UDP:104 ICMP:0 Other:33>


=====================D-LinkSiren\Setup-C-5-STA.pcap=====================

<Setup-C-5-STA.pcap: TCP:465 UDP:102 ICMP:0 Other:31>


=====================D-LinkSiren\Setup-C-6-STA.pcap=====================

<Setup-C-6-STA.pcap: TCP:462 UDP:115 ICMP:0 Other:28>


=====================D-LinkSiren\Setup-C-7-STA.pcap=====================

<Setup-C-7-S



=====================EdimaxCam2\Setup-A-2-STA.pcap=====================

<Setup-A-2-STA.pcap: TCP:0 UDP:34 ICMP:0 Other:8>


=====================EdimaxCam2\Setup-A-3-STA.pcap=====================

<Setup-A-3-STA.pcap: TCP:0 UDP:39 ICMP:0 Other:13>


=====================EdimaxCam2\Setup-A-4-STA.pcap=====================

<Setup-A-4-STA.pcap: TCP:0 UDP:33 ICMP:0 Other:8>


=====================EdimaxCam2\Setup-C-1-STA.pcap=====================

<Setup-C-1-STA.pcap: TCP:0 UDP:37 ICMP:0 Other:8>


=====================EdimaxCam2\Setup-C-2-STA.pcap=====================

<Setup-C-2-STA.pcap: TCP:0 UDP:39 ICMP:0 Other:16>


=====================EdimaxCam2\Setup-C-3-STA.pcap=====================

<Setup-C-3-STA.pcap: TCP:0 UDP:33 ICMP:0 Other:9>


=====================EdimaxCam2\Setup-C-4-STA.pcap=====================

<Setup-C-4-STA.pcap: TCP:0 UDP:39 ICMP:0 Other:13>


=====================EdimaxPlug1101W\Setup-A-1-STA.pcap=====================

<Setup-A-1-STA.pcap: TCP:41 UDP:41 ICMP:0 



=====================EdnetCam2\Setup-C-6-STA.pcap=====================

<Setup-C-6-STA.pcap: TCP:0 UDP:14 ICMP:1 Other:10>


=====================EdnetCam2\Setup-C-7-STA.pcap=====================

<Setup-C-7-STA.pcap: TCP:0 UDP:14 ICMP:0 Other:10>


=====================EdnetCam2\Setup-C-8-STA.pcap=====================

<Setup-C-8-STA.pcap: TCP:0 UDP:5 ICMP:0 Other:8>


=====================EdnetGateway\Setup-A-1-STA.pcap=====================

<Setup-A-1-STA.pcap: TCP:30 UDP:23 ICMP:5 Other:12>


=====================EdnetGateway\Setup-A-2-STA.pcap=====================

<Setup-A-2-STA.pcap: TCP:35 UDP:24 ICMP:6 Other:18>


=====================EdnetGateway\Setup-C-1-STA.pcap=====================

<Setup-C-1-STA.pcap: TCP:32 UDP:22 ICMP:4 Other:16>


=====================EdnetGateway\Setup-C-10-STA.pcap=====================

<Setup-C-10-STA.pcap: TCP:31 UDP:21 ICMP:3 Other:10>


=====================EdnetGateway\Setup-C-11-STA.pcap=====================

<Setup-C-11-STA.pcap: TCP:30 UD



=====================HueSwitch\Setup-A-10-STA.pcap=====================

<Setup-A-10-STA.pcap: TCP:2129 UDP:28 ICMP:0 Other:2>


=====================HueSwitch\Setup-A-2-STA.pcap=====================

<Setup-A-2-STA.pcap: TCP:2302 UDP:32 ICMP:0 Other:4>


=====================HueSwitch\Setup-A-3-STA.pcap=====================

<Setup-A-3-STA.pcap: TCP:2100 UDP:44 ICMP:0 Other:4>


=====================HueSwitch\Setup-A-4-STA.pcap=====================

<Setup-A-4-STA.pcap: TCP:2064 UDP:38 ICMP:0 Other:6>


=====================HueSwitch\Setup-A-5-STA.pcap=====================

<Setup-A-5-STA.pcap: TCP:1314 UDP:42 ICMP:0 Other:4>


=====================HueSwitch\Setup-A-6-STA.pcap=====================

<Setup-A-6-STA.pcap: TCP:2133 UDP:54 ICMP:0 Other:4>


=====================HueSwitch\Setup-A-7-STA.pcap=====================

<Setup-A-7-STA.pcap: TCP:1693 UDP:22 ICMP:0 Other:2>


=====================HueSwitch\Setup-A-8-STA.pcap=====================

<Setup-A-8-STA.pcap: TCP:1692 UDP:2



=====================MAXGateway\Setup-A-8-STA.pcap=====================

<Setup-A-8-STA.pcap: TCP:52 UDP:4 ICMP:0 Other:6>


=====================MAXGateway\Setup-A-9-STA.pcap=====================

<Setup-A-9-STA.pcap: TCP:48 UDP:10 ICMP:0 Other:11>


=====================MAXGateway\Setup-C-1-STA.pcap=====================

<Setup-C-1-STA.pcap: TCP:40 UDP:7 ICMP:1 Other:4>


=====================MAXGateway\Setup-C-10-STA.pcap=====================

<Setup-C-10-STA.pcap: TCP:40 UDP:5 ICMP:0 Other:7>


=====================MAXGateway\Setup-C-11-STA.pcap=====================

<Setup-C-11-STA.pcap: TCP:64 UDP:5 ICMP:0 Other:9>


=====================MAXGateway\Setup-C-2-STA.pcap=====================

<Setup-C-2-STA.pcap: TCP:44 UDP:9 ICMP:0 Other:7>


=====================MAXGateway\Setup-C-3-STA.pcap=====================

<Setup-C-3-STA.pcap: TCP:44 UDP:5 ICMP:0 Other:7>


=====================MAXGateway\Setup-C-4-STA.pcap=====================

<Setup-C-4-STA.pcap: TCP:40 UDP:5 ICMP:0 Oth



=====================TP-LinkPlugHS110\Setup-C-2-STA.pcap=====================

<Setup-C-2-STA.pcap: TCP:40 UDP:10 ICMP:0 Other:7>


=====================TP-LinkPlugHS110\Setup-C-3-STA.pcap=====================

<Setup-C-3-STA.pcap: TCP:44 UDP:10 ICMP:0 Other:8>


=====================TP-LinkPlugHS110\Setup-C-4-STA.pcap=====================

<Setup-C-4-STA.pcap: TCP:43 UDP:19 ICMP:0 Other:7>


=====================TP-LinkPlugHS110\Setup-C-5-STA.pcap=====================

<Setup-C-5-STA.pcap: TCP:40 UDP:10 ICMP:0 Other:7>


=====================TP-LinkPlugHS110\Setup-C-6-STA.pcap=====================

<Setup-C-6-STA.pcap: TCP:43 UDP:10 ICMP:0 Other:7>


=====================TP-LinkPlugHS110\Setup-C-7-STA.pcap=====================

<Setup-C-7-STA.pcap: TCP:43 UDP:10 ICMP:0 Other:8>


=====================TP-LinkPlugHS110\Setup-C-8-STA.pcap=====================

<Setup-C-8-STA.pcap: TCP:44 UDP:10 ICMP:0 Other:7>


=====================TP-LinkPlugHS110\Setup-C-9-STA.pcap==================



=====================WeMoSwitch\Setup-C-1-STA.pcap=====================

<Setup-C-1-STA.pcap: TCP:235 UDP:24 ICMP:10 Other:15>


=====================WeMoSwitch\Setup-C-10-STA.pcap=====================

<Setup-C-10-STA.pcap: TCP:239 UDP:24 ICMP:10 Other:14>


=====================WeMoSwitch\Setup-C-2-STA.pcap=====================

<Setup-C-2-STA.pcap: TCP:301 UDP:26 ICMP:10 Other:14>


=====================WeMoSwitch\Setup-C-3-STA.pcap=====================

<Setup-C-3-STA.pcap: TCP:207 UDP:25 ICMP:11 Other:19>


=====================WeMoSwitch\Setup-C-4-STA.pcap=====================

<Setup-C-4-STA.pcap: TCP:236 UDP:24 ICMP:10 Other:14>


=====================WeMoSwitch\Setup-C-5-STA.pcap=====================

<Setup-C-5-STA.pcap: TCP:241 UDP:24 ICMP:10 Other:15>


=====================WeMoSwitch\Setup-C-6-STA.pcap=====================

<Setup-C-6-STA.pcap: TCP:237 UDP:24 ICMP:10 Other:15>


=====================WeMoSwitch\Setup-C-7-STA.pcap=====================

<Setup-C-7-STA.pcap:

Below, csv files created from pcap files are organized for use in machine learning methods.


In this section, all csv files in the `csvs` folder are found and their addresses are assigned to the variable named files_add.
These csv files were created through the top block of code.
# &darr;  

In [21]:
def find_the_way(path,file_format):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                files_add.append(os.path.join(r, file))  
    return files_add
files_add=find_the_way('./csvs/','.csv')


In this section, files with the same device label are combined to create one file for each device. For example, 20 different data collected from the Aria device are collected in a single file:`Aria_all.csv`.
In addition, a header describing the features is placed at the beginning of the files.
The created file example can be found [here](https://github.com/bozbil/heriot-watt/blob/master/fingerprint/Aria_all.csv).

In [22]:
features_name=['ARP',
    'LLC',
    'EAPOL',
    "IP",
    'ICMP',
    'ICMP6',
    'TCP',
    'UDP',
    'TCP_w_size',
    'HTTP',
    'HTTPS',
    'DHCP',
    'BOOTP',
    'SSDP',
    'DNS',
    'MDNS',
    'NTP',
    'IP_padding',
    'IP_add_count',
    'IP_ralert',
    'Portcl_src',
    'Portcl_dst',
    'Pck_size',
    'Pck_rawdata',
    "Entropy",
    'Label']             
   
features_name=( ",".join( i for i in features_name ) )
devices=[]
flag=False
for i in files_add:
    temp=i
    where_is_at=temp.find("@")
    temp=i[7:where_is_at]
    if temp in devices:
        name=temp+"_all.csv"
        ths = open(name, "a") 
        with open(i, "r") as file:
            while True:   
                line=file.readline()
                if line=="":
                    break
                else:
                    line=line
                    ths.write(str(line))
    else:
        devices.append(temp)
        if flag:
            ths.close()
        name=temp+"_all.csv"
        ths = open(name, "w") 
        ths.write(features_name)
        with open(i, "r") as file:
            while True:   
                line=file.readline()
                if line=="":
                    break
                else:
                    line=line
                    ths.write(str(line))
        flag=True 
ths.close()        

In this section,all csv files created are merged under a single file:`_all.csv`.
In addition, a header describing the features is placed at the beginning of the files.
The created file example can be found [here](https://github.com/bozbil/heriot-watt/blob/master/fingerprint/_all.csv).

In [23]:
name="hybrid.csv"
ths = open(name, "w") 
ths.write(features_name+"\n")
for i in files_add:
    with open(i, "r") as file:
        while True:   
            line=file.readline()
            if line=="":
                break
            else:
                line=line
                ths.write(str(line))
ths.close()     
    
    
    
    

       